In [1]:
# Loading up libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Loading up csv
# df = pd.read_csv(r"./csv/data_3600.csv")
# df = pd.read_csv(r"./csv/data_36000.csv")
df = pd.read_csv(r"./csv/data_all.csv")
pd.set_option('display.max_rows', 500)

In [3]:
SMAL = df['Price'].rolling(window = 2).mean()
SMAH = df['Price'].rolling(window = 22).mean()
df["SMAL"] = SMAL
df["SMAH"] = SMAH

In [4]:
def checkCrossover(row):
    if row["SMAL"] > row["SMAH"]:
        return "Higher"
    else:
        return "Lower"

In [5]:
df["Crossover"] = df.apply(checkCrossover, axis=1)

In [6]:
df["Change"] = ""
df["isBuy"] = ""
df["TimeLimit"] = 0
df["Action"] = "---"
df["Profit"] = "---"

In [7]:

report = {
    "buy":{
        "time":0,
        "price":0
    },
    "sell":{
        "time":0,
        "price":0
    },
    "profit": 0
}

for index, row in df.iterrows():
    if index == 0:
        df["Change"][index] = "N/A"
        df["isBuy"][index] = True
        df["TimeLimit"][index] = 0
        df["Action"][index] = "Buy!"
        df["Profit"][index] = float(df["Price"][index])
        __currentMode = "Buy!"
        __currentProfit = df["Profit"][index]
        report["buy"]["price"] = df["Price"][index]


    else:
            df["TimeLimit"][index] = df["TimeLimit"][index - 1] + 1
        
            __crossOverPrev = df["Crossover"][index - 1]
            __crossOverPres = df["Crossover"][index]
            __timeLimit = df["TimeLimit"][index]

            if __crossOverPrev != __crossOverPres:
                df["Change"][index] = "Crossed"
            else:
                df["Change"][index] = "---"
            
            
            if df["Crossover"][index] == "Lower":
                df["isBuy"][index] = True
            else:
                df["isBuy"][index] = False           
            
            
            
            df["TimeLimit"][index] = __timeLimit                        
                    
            if df["Change"][index] == "Crossed" and df["isBuy"][index] and __currentMode != "Buy!":
                df["Action"][index] = "Buy!"
                __currentMode = "Buy!"
                df["Profit"][index] = df["Price"][index]
                report["buy"]["time"] = df["Time"][index]
                report["buy"]["price"] = df["Price"][index]
                df["TimeLimit"][index] = 0 
                
            if  df["Price"][index] < report["buy"]["price"] and df["Change"][index] == "Crossed":
                df["Change"][index + 1] == "Crossed"
                
            if  df["Price"][index] >= report["buy"]["price"] and df["TimeLimit"][index] in range(30,59 + 1) and df["Change"][index] == "Crossed" and not df["isBuy"][index] and __currentMode != "Sell!":
                df["Action"][index] = "Sell!"
                __currentMode = "Sell!"                    
                df["Profit"][index] = df["Price"][index]
                report["sell"]["time"] = df["Time"][index]
                report["sell"]["price"] = df["Price"][index]
                t_profit = round(report["sell"]["price"] -  report["buy"]["price"],4)
                report["profit"] +=  t_profit
                print(f'Open at {report["buy"]["time"]}({report["buy"]["price"]}) | Close at {report["sell"]["time"]}({report["sell"]["price"]}) | for profit ({t_profit}) < {df["TimeLimit"][index]}mins >')

            elif df["TimeLimit"][index] >= 59 and __currentMode == "Buy!":
                df["Action"][index] = "Sell!"
                __currentMode = "Sell!"
                df["Profit"][index] = df["Price"][index]
                report["sell"]["time"] = df["Time"][index]
                report["sell"]["price"] = df["Price"][index]
                t_profit = round(report["sell"]["price"] -  report["buy"]["price"],4)
                report["profit"] +=  report["sell"]["price"] -  report["buy"]["price"]
                print(f'Open at {report["buy"]["time"]}({report["buy"]["price"]}) | Close at {report["sell"]["time"]}({report["sell"]["price"]}) | for profit ({t_profit}) < {df["TimeLimit"][index]}mins > -- CLOSED' )
print(f'\nTotal profit: {report["profit"]}')

Open at 0(1.2546) | Close at 46(1.2629) | for profit (0.0083) < 46mins >
Open at 64(1.2635) | Close at 106(1.2832) | for profit (0.0197) < 42mins >
Open at 107(1.2769) | Close at 137(1.2914) | for profit (0.0145) < 30mins >
Open at 139(1.2846) | Close at 175(1.3001) | for profit (0.0155) < 36mins >
Open at 177(1.2966) | Close at 236(1.3661) | for profit (0.0695) < 59mins > -- CLOSED
Open at 247(1.3574) | Close at 278(1.36484) | for profit (0.0074) < 31mins >
Open at 280(1.36192) | Close at 339(1.35833) | for profit (-0.0036) < 59mins > -- CLOSED
Open at 341(1.35796) | Close at 400(1.32012) | for profit (-0.0378) < 59mins > -- CLOSED
Open at 402(1.31882) | Close at 437(1.32568) | for profit (0.0069) < 35mins >
Open at 442(1.32212) | Close at 483(1.34602) | for profit (0.0239) < 41mins >
Open at 485(1.34303) | Close at 544(1.32807) | for profit (-0.015) < 59mins > -- CLOSED
Open at 581(1.32425) | Close at 640(1.33745) | for profit (0.0132) < 59mins > -- CLOSED
Open at 647(1.32992) | Clos

Open at 6612(1.35502) | Close at 6671(1.33982) | for profit (-0.0152) < 59mins > -- CLOSED
Open at 6685(1.33508) | Close at 6744(1.3631) | for profit (0.028) < 59mins > -- CLOSED
Open at 6758(1.35836) | Close at 6806(1.36362) | for profit (0.0053) < 48mins >
Open at 6808(1.36264) | Close at 6867(1.34612) | for profit (-0.0165) < 59mins > -- CLOSED
Open at 6895(1.34659) | Close at 6954(1.3321) | for profit (-0.0145) < 59mins > -- CLOSED
Open at 6993(1.3327) | Close at 7052(1.32002) | for profit (-0.0127) < 59mins > -- CLOSED
Open at 7082(1.32292) | Close at 7141(1.30559) | for profit (-0.0173) < 59mins > -- CLOSED
Open at 7206(1.26634) | Close at 7264(1.27393) | for profit (0.0076) < 58mins >
Open at 7265(1.26941) | Close at 7324(1.25307) | for profit (-0.0163) < 59mins > -- CLOSED
Open at 7358(1.22988) | Close at 7403(1.2313) | for profit (0.0014) < 45mins >
Open at 7425(1.2333) | Close at 7484(1.23491) | for profit (0.0016) < 59mins > -- CLOSED
Open at 7517(1.22859) | Close at 7565(1.

Open at 13465(1.43305) | Close at 13524(1.4169) | for profit (-0.0161) < 59mins > -- CLOSED
Open at 13538(1.41973) | Close at 13595(1.4203) | for profit (0.0006) < 57mins >
Open at 13597(1.41409) | Close at 13631(1.42534) | for profit (0.0112) < 34mins >
Open at 13641(1.42191) | Close at 13700(1.40461) | for profit (-0.0173) < 59mins > -- CLOSED
Open at 13703(1.40122) | Close at 13740(1.40718) | for profit (0.006) < 37mins >
Open at 13741(1.40521) | Close at 13776(1.41581) | for profit (0.0106) < 35mins >
Open at 13805(1.42648) | Close at 13860(1.44133) | for profit (0.0149) < 55mins >
Open at 13864(1.43918) | Close at 13923(1.46382) | for profit (0.0246) < 59mins > -- CLOSED
Open at 13929(1.46055) | Close at 13988(1.45776) | for profit (-0.0028) < 59mins > -- CLOSED
Open at 14003(1.45331) | Close at 14062(1.44156) | for profit (-0.0118) < 59mins > -- CLOSED
Open at 14084(1.44427) | Close at 14143(1.41694) | for profit (-0.0273) < 59mins > -- CLOSED
Open at 14161(1.42276) | Close at 14

Open at 19981(1.25428) | Close at 20040(1.25337) | for profit (-0.0009) < 59mins > -- CLOSED
Open at 20047(1.25362) | Close at 20106(1.23594) | for profit (-0.0177) < 59mins > -- CLOSED
Open at 20164(1.2447) | Close at 20194(1.25103) | for profit (0.0063) < 30mins >
Open at 20220(1.25664) | Close at 20279(1.24821) | for profit (-0.0084) < 59mins > -- CLOSED
Open at 20290(1.24777) | Close at 20337(1.258) | for profit (0.0102) < 47mins >
Open at 20360(1.26036) | Close at 20400(1.26116) | for profit (0.0008) < 40mins >
Open at 20439(1.26658) | Close at 20498(1.25236) | for profit (-0.0142) < 59mins > -- CLOSED
Open at 20526(1.25072) | Close at 20585(1.24431) | for profit (-0.0064) < 59mins > -- CLOSED
Open at 20614(1.26252) | Close at 20673(1.25613) | for profit (-0.0064) < 59mins > -- CLOSED
Open at 20759(1.22899) | Close at 20818(1.21855) | for profit (-0.0104) < 59mins > -- CLOSED
Open at 20840(1.2192) | Close at 20895(1.22862) | for profit (0.0094) < 55mins >
Open at 20910(1.22367) | 

Open at 26500(1.33236) | Close at 26534(1.33412) | for profit (0.0018) < 34mins >
Open at 26545(1.33487) | Close at 26575(1.33998) | for profit (0.0051) < 30mins >
Open at 26577(1.339) | Close at 26636(1.31054) | for profit (-0.0285) < 59mins > -- CLOSED
Open at 26671(1.30953) | Close at 26730(1.30334) | for profit (-0.0062) < 59mins > -- CLOSED
Open at 26746(1.29988) | Close at 26805(1.29725) | for profit (-0.0026) < 59mins > -- CLOSED
Open at 26835(1.29965) | Close at 26894(1.28705) | for profit (-0.0126) < 59mins > -- CLOSED
Open at 26899(1.28543) | Close at 26958(1.30146) | for profit (0.016) < 59mins > -- CLOSED
Open at 26959(1.30234) | Close at 26989(1.30681) | for profit (0.0045) < 30mins >
Open at 27000(1.30548) | Close at 27049(1.31503) | for profit (0.0095) < 49mins >
Open at 27052(1.31303) | Close at 27111(1.3158) | for profit (0.0028) < 59mins > -- CLOSED
Open at 27142(1.31839) | Close at 27174(1.32341) | for profit (0.005) < 32mins >
Open at 27176(1.32057) | Close at 27225

Open at 32632(1.35414) | Close at 32678(1.35752) | for profit (0.0034) < 46mins >
Open at 32680(1.35648) | Close at 32739(1.36063) | for profit (0.0042) < 59mins > -- CLOSED
Open at 32749(1.361) | Close at 32796(1.36105) | for profit (0.0001) < 47mins >
Open at 32805(1.35874) | Close at 32862(1.36183) | for profit (0.0031) < 57mins >
Open at 32872(1.36112) | Close at 32931(1.36792) | for profit (0.0068) < 59mins > -- CLOSED
Open at 32966(1.36519) | Close at 33025(1.36055) | for profit (-0.0046) < 59mins > -- CLOSED
Open at 33033(1.36006) | Close at 33066(1.3639) | for profit (0.0038) < 33mins >
Open at 33083(1.36342) | Close at 33142(1.36189) | for profit (-0.0015) < 59mins > -- CLOSED
Open at 33149(1.3614) | Close at 33208(1.35261) | for profit (-0.0088) < 59mins > -- CLOSED
Open at 33209(1.35175) | Close at 33267(1.35254) | for profit (0.0008) < 58mins >
Open at 33272(1.35196) | Close at 33331(1.34663) | for profit (-0.0053) < 59mins > -- CLOSED
Open at 33332(1.34629) | Close at 3339

Open at 39535(1.1074) | Close at 39594(1.0923) | for profit (-0.0151) < 59mins > -- CLOSED
Open at 39623(1.09822) | Close at 39682(1.08726) | for profit (-0.011) < 59mins > -- CLOSED
Open at 39691(1.08674) | Close at 39731(1.09398) | for profit (0.0072) < 40mins >
Open at 39736(1.08586) | Close at 39779(1.10371) | for profit (0.0178) < 43mins >
Open at 39823(1.11419) | Close at 39853(1.11499) | for profit (0.0008) < 30mins >
Open at 39857(1.11373) | Close at 39916(1.10373) | for profit (-0.01) < 59mins > -- CLOSED
Open at 39928(1.10366) | Close at 39987(1.1376) | for profit (0.0339) < 59mins > -- CLOSED
Open at 40014(1.15322) | Close at 40073(1.12521) | for profit (-0.028) < 59mins > -- CLOSED
Open at 40096(1.12441) | Close at 40155(1.12949) | for profit (0.0051) < 59mins > -- CLOSED
Open at 40157(1.12638) | Close at 40216(1.1129) | for profit (-0.0135) < 59mins > -- CLOSED
Open at 40217(1.11186) | Close at 40265(1.11764) | for profit (0.0058) < 48mins >
Open at 40279(1.11741) | Close 

Open at 45954(1.10759) | Close at 46011(1.11777) | for profit (0.0102) < 57mins >
Open at 46013(1.1157) | Close at 46052(1.11733) | for profit (0.0016) < 39mins >
Open at 46054(1.11656) | Close at 46111(1.12897) | for profit (0.0124) < 57mins >
Open at 46144(1.13328) | Close at 46203(1.13224) | for profit (-0.001) < 59mins > -- CLOSED
Open at 46276(1.12366) | Close at 46335(1.11593) | for profit (-0.0077) < 59mins > -- CLOSED
Open at 46339(1.11369) | Close at 46371(1.11907) | for profit (0.0054) < 32mins >
Open at 46389(1.11826) | Close at 46448(1.12477) | for profit (0.0065) < 59mins > -- CLOSED
Open at 46464(1.12371) | Close at 46500(1.12677) | for profit (0.0031) < 36mins >
Open at 46511(1.12616) | Close at 46570(1.12156) | for profit (-0.0046) < 59mins > -- CLOSED
Open at 46581(1.12239) | Close at 46612(1.1245) | for profit (0.0021) < 31mins >
Open at 46622(1.12414) | Close at 46681(1.11774) | for profit (-0.0064) < 59mins > -- CLOSED
Open at 46682(1.11789) | Close at 46741(1.1206)

Open at 52557(1.19909) | Close at 52616(1.18948) | for profit (-0.0096) < 59mins > -- CLOSED
Open at 52617(1.18846) | Close at 52661(1.19056) | for profit (0.0021) < 44mins >
Open at 52666(1.18787) | Close at 52706(1.1926) | for profit (0.0047) < 40mins >
Open at 52707(1.19224) | Close at 52766(1.19978) | for profit (0.0075) < 59mins > -- CLOSED
Open at 52788(1.19614) | Close at 52847(1.19187) | for profit (-0.0043) < 59mins > -- CLOSED
Open at 52871(1.19233) | Close at 52930(1.19992) | for profit (0.0076) < 59mins > -- CLOSED
Open at 52935(1.1992) | Close at 52994(1.19266) | for profit (-0.0065) < 59mins > -- CLOSED
Open at 53096(1.17742) | Close at 53155(1.1753) | for profit (-0.0021) < 59mins > -- CLOSED
Open at 53178(1.17611) | Close at 53237(1.17412) | for profit (-0.002) < 59mins > -- CLOSED
Open at 53242(1.1728) | Close at 53301(1.18539) | for profit (0.0126) < 59mins > -- CLOSED
Open at 53316(1.18522) | Close at 53375(1.17958) | for profit (-0.0056) < 59mins > -- CLOSED
Open at

Open at 59309(1.15987) | Close at 59368(1.14833) | for profit (-0.0115) < 59mins > -- CLOSED
Open at 59398(1.15086) | Close at 59457(1.14806) | for profit (-0.0028) < 59mins > -- CLOSED
Open at 59475(1.14881) | Close at 59534(1.15364) | for profit (0.0048) < 59mins > -- CLOSED
Open at 59564(1.15727) | Close at 59623(1.14914) | for profit (-0.0081) < 59mins > -- CLOSED
Open at 59667(1.15027) | Close at 59726(1.13919) | for profit (-0.0111) < 59mins > -- CLOSED
Open at 59744(1.13793) | Close at 59788(1.13939) | for profit (0.0015) < 44mins >
Open at 59789(1.13858) | Close at 59848(1.13143) | for profit (-0.0071) < 59mins > -- CLOSED
Open at 59888(1.13774) | Close at 59932(1.14177) | for profit (0.004) < 44mins >
Open at 59962(1.14494) | Close at 60021(1.13094) | for profit (-0.014) < 59mins > -- CLOSED
Open at 60048(1.1235) | Close at 60078(1.13331) | for profit (0.0098) < 30mins >
Open at 60097(1.1285) | Close at 60156(1.14508) | for profit (0.0166) < 59mins > -- CLOSED
Open at 60168(1.

In [8]:
df[:250]

,Time,Price,SMAL,SMAH,Crossover,Change,isBuy,TimeLimit,Action,Profit
0,0,1.2546,NaN,NaN,Lower,N/A,True,0,Buy!,1.2546
1,1,1.2567,1.25565,NaN,Lower,---,True,1,---,---
2,2,1.2577,1.25720,NaN,Lower,---,True,2,---,---
3,3,1.2579,1.25780,NaN,Lower,---,True,3,---,---
4,4,1.2593,1.25860,NaN,Lower,---,True,4,---,---
5,5,1.2668,1.26305,NaN,Lower,---,True,5,---,---
6,6,1.2695,1.26815,NaN,Lower,---,True,6,---,---
7,7,1.2689,1.26920,NaN,Lower,---,True,7,---,---
8,8,1.2679,1.26840,NaN,Lower,---,True,8,---,---
9,9,1.2679,1.26790,NaN,Lower,---,True,9,---,---


In [9]:
buy = df[df["Action"] == "Buy!"]["Price"].sum()
df[df["Action"] == "Buy!"]["Price"].count()
# t.groupBy("Price").sum()

940

In [10]:
sell = df[df["Action"] == "Sell!"]["Price"].sum()
df[df["Action"] == "Sell!"]["Price"].count()

940

In [11]:
sell - buy

0.12958000000003267

In [12]:
# buy = df[df["Action"] == "Buy!"]["Price"]
# sell = df[df["Action"] == "Sell!"]["Price"]

In [13]:
# for index, row in buy.iterrows():
#     print(buy[index])